## Reinforcement Learning Tutorial

In [1]:
pip install gym==0.17.3


  Using cached pyglet-1.5.0-py2.py3-none-any.whl.metadata (7.6 kB)
Using cached pyglet-1.5.0-py2.py3-none-any.whl (1.0 MB)
  Attempting uninstall: pyglet
    Found existing installation: pyglet 1.5.28
    Uninstalling pyglet-1.5.28:
      Successfully uninstalled pyglet-1.5.28

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pyglet==1.5.0


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Some Common Terms
- Agent
- Environment
- Actions, Rewards, Observations

In [3]:
pip install --user --upgrade git+http://github.com/pyglet/pyglet@pyglet-1.5-maintenance

  Cloning http://github.com/pyglet/pyglet (to revision pyglet-1.5-maintenance) to /private/var/folders/7s/1h5g7ghd0j57351mn1qdyg9c0000gn/T/pip-req-build-frqkyp7s
  Running command git clone --filter=blob:none --quiet http://github.com/pyglet/pyglet /private/var/folders/7s/1h5g7ghd0j57351mn1qdyg9c0000gn/T/pip-req-build-frqkyp7s
  Running command git checkout -b pyglet-1.5-maintenance --track origin/pyglet-1.5-maintenance
  Switched to a new branch 'pyglet-1.5-maintenance'
  branch 'pyglet-1.5-maintenance' set up to track 'origin/pyglet-1.5-maintenance'.
  Resolved http://github.com/pyglet/pyglet to commit a728a77cfa94c917c0a375e73e576eac57ffffbf
  Preparing metadata (setup.py) ... done
  Created wheel for pyglet: filename=pyglet-1.5.28-py3-none-any.whl size=1143399 sha256=6046af88f0d655cc4618f66253b038db3ec33ba6cdfc247967d65a2d0ace3775
  Stored in directory: /private/var/folders/7s/1h5g7ghd0j57351mn1qdyg9c0000gn/T/pip-ephem-wheel-cache-rvoqngek/wheels/2c/22/26/4717af2e23b7db72f1e7384a81

### 1. Interacting with the Gym API

In [13]:
import gym

In [14]:
# Create Enviroment 
env = gym.make('CartPole-v0')


#### Each environment comes with certain import methods/attributes
- action_space
- observation_space
- reset() : returns init state and also resets the environment
- step()
- render()

In [6]:
env.reset()

array([ 0.04727144,  0.01228832, -0.03411722,  0.01700545])

In [5]:
for t in range(1000):
    env.render()

env.close()

2024-03-10 09:04:04.112 Python[5354:225153] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/7s/1h5g7ghd0j57351mn1qdyg9c0000gn/T/org.python.python.savedState


In [7]:
pip install pygame


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
env.action_space

Discrete(2)

In [9]:
env.action_space.n

2

In [10]:
env.observation_space.shape[0]

4

In [11]:
env.reset()
for t in range(1000):
    random_action = env.action_space.sample()
    done = env.step(random_action) #randomly move left or right
    if(done):
        break
    env.render()
env.close()

: 

### 2. Playing Games with a Random Strategy
- Game Episode
- Step() Function in More Detail
- Game Over?


In [7]:
for e in range(20): #Episode
    #Play 20 episodes 
    observation = env.reset()
    for t in range(50):
        env.render()
        action = env.action_space.sample()
        observation,reward,done,other_info = env.step(action)
        
        if done:
            # Game Episode is over
            print("Game Episode :{}/{} High Score :{}".format(e,20,t))
            break

env.close()
print("All 20 episodes over!")


2024-03-10 09:19:55.551 Python[5538:235452] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/7s/1h5g7ghd0j57351mn1qdyg9c0000gn/T/org.python.python.savedState


Game Episode :0/20 High Score :20
Game Episode :1/20 High Score :46
Game Episode :2/20 High Score :14
Game Episode :3/20 High Score :11
Game Episode :4/20 High Score :21
Game Episode :5/20 High Score :19
Game Episode :6/20 High Score :15
Game Episode :7/20 High Score :11
Game Episode :8/20 High Score :10
Game Episode :9/20 High Score :22
Game Episode :10/20 High Score :15
Game Episode :11/20 High Score :20
Game Episode :12/20 High Score :15
Game Episode :13/20 High Score :22
Game Episode :14/20 High Score :14
Game Episode :15/20 High Score :12
Game Episode :16/20 High Score :22
Game Episode :17/20 High Score :12
Game Episode :18/20 High Score :16
Game Episode :19/20 High Score :25
All 20 episodes over!


### 3. Q-Learning

Designing an AI Agent

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import random

# Create Enviroment
import gym
env = gym.make('CartPole-v0')


2024-03-10 14:34:38.982480: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

class Agent:
    def __init__(self,state_size,action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95 #Discount Factor
        self.epsilon = 1.0 # Exploration Rate: How much to act randomly, 
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.learning_rate = 0.001 
        self.model = self._create_model()
        
    
    def _create_model(self):
        #Neural Network To Approximate Q-Value function
        model = Sequential()
        model.add(Dense(24,input_dim=self.state_size,activation='relu')) #1st Hidden Layer
        model.add(Dense(24,activation='relu')) #2nd Hidden Layer
        model.add(Dense(self.action_size,activation='linear'))
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done)) #remembering previous experiences
        
    def act(self,state):
        # Exploration vs Exploitation
        if np.random.rand()<=self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state) # predict reward value based upon current state
        return np.argmax(act_values[0]) #Left or Right
    
    def train(self,batch_size=32): #method that trains NN with experiences sampled from memory
        minibatch = random.sample(self.memory,batch_size)
        for state,action,reward,next_state,done in minibatch:
            
            if not done: #boolean 
                target = reward + self.gamma*np.amax(self.model.predict(next_state)[0])
            else:
                target = reward
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state,target_f,epochs=1,verbose=0) #single epoch, x =state, y = target_f, loss--> target_f - 
            
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay # -----> reduce exploration on end of train of batch_size
    
    def load(self,name):
        self.model.load_weights(name)
    def save(self,name):
        self.model.save_weights(name)
            

In [3]:
np.random.rand()

0.5445426028961114

#### Training the DQN Agent (Deep Q-Learner)

In [4]:
n_episodes = 1000
output_dir = "cartpole_model/"

In [5]:
agent = Agent(state_size=4,action_size=2)
done = False
state_size = 4
action_size =2
batch_size = 32

In [6]:
agent = Agent(state_size, action_size) # initialise agent
done = False
for e in range(n_episodes):
    state = env.reset()
    state = np.reshape(state,[1,state_size])
    # -----> Start with a new game
    for time in range(5000):
        env.render() # -----> Renders the environments to help visualise what the agent see
        action = agent.act(state) #action is 0 or 1 # -----> Select a random action
        next_state,reward,done,other_info = env.step(action) # -----> Take a random action and env gets into a new state - env gives the reward.
        reward = reward if not done else -10 # -----> 
        next_state = np.reshape(next_state,[1,state_size])
        agent.remember(state,action,reward,next_state,done) # -----> Agent remembers this one experience in replay buffer
        state = next_state
        
        if done:
            print("Game Episode :{}/{}, High Score:{},Exploration Rate:{:.2}".format(e,n_episodes,time,agent.epsilon))
            break
    
    if len(agent.memory)>batch_size:# -----> when you've sufficient experiences in the Agent buffer, start sampling from it, start training neural network
        agent.train(batch_size)
    
    if e%50==0:
        agent.save(output_dir+"weights_"+'{:04d}'.format(e)+".hdf5")
        
env.close()

2024-03-10 14:34:56.351 Python[3506:121690] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/7s/1h5g7ghd0j57351mn1qdyg9c0000gn/T/org.python.python.savedState


Game Episode :0/1000, High Score:8,Exploration Rate:1.0
Game Episode :1/1000, High Score:16,Exploration Rate:1.0
Game Episode :2/1000, High Score:17,Exploration Rate:1.0
1/1 [==============================] - 0s 19ms/step
Game Episode :3/1000, High Score:19,Exploration Rate:0.99
1/1 [==============================] - 0s 18ms/step
Game Episode :4/1000, High Score:23,Exploration Rate:0.99
1/1 [==============================] - 0s 18ms/step
Game Episode :5/1000, High Score:10,Exploration Rate:0.99
1/1 [==============================] - 0s 19ms/step


2024-03-10 14:35:18.146 Python[3506:121690] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


Game Episode :6/1000, High Score:33,Exploration Rate:0.98
1/1 [==============================] - 0s 19ms/step
Game Episode :7/1000, High Score:10,Exploration Rate:0.98
1/1 [==============================] - 0s 19ms/step
Game Episode :8/1000, High Score:16,Exploration Rate:0.97
1/1 [==============================] - 0s 19ms/step
Game Episode :9/1000, High Score:16,Exploration Rate:0.97
1/1 [==============================] - 0s 21ms/step
Game Episode :10/1000, High Score:13,Exploration Rate:0.96
1/1 [==============================] - 0s 22ms/step
Game Episode :11/1000, High Score:7,Exploration Rate:0.96
1/1 [==============================] - 0s 16ms/step
Game Episode :12/1000, High Score:22,Exploration Rate:0.95
1/1 [==============================] - 0s 19ms/step
Game Episode :13/1000, High Score:33,Exploration Rate:0.95
1/1 [==============================] - 0s 18ms/step
Game Episode :14/1000, High Score:10,Exploration Rate:0.94
1/1 [==============================] - 0s 19ms/step
Game E

KeyboardInterrupt: 